### 流程

1.天气数据类 & 天气数据容器类
2.用户交互类
3.事件类
4.预测模型类
5.事件决策类


In [1]:
# 定义事件类
# 定义用户事件
class Event(object):
    """ pass  """

    def __init__(self, name, outdoors, cover_available, time):
        """
        Parameters:
            name (str): pass .
            outdoors (bool): pass.
            cover_available (bool): pass .
            time (int): pass .
        """
        self._name = name
        self._outdoors = outdoors
        self._cover_available = cover_available
        self._time = time

    def get_name(self):
        """(str) """
        return self._name

    def get_time(self):
        """(int) """
        return self._time

    def get_outdoors(self):
        """(bool) """
        return self._outdoors

    def get_cover_available(self):
        """(bool) """
        return self._cover_available

    def __str__(self):
        """(str) """
        return f"Event({self._name} @ {self._time}, {self._outdoors}, {self.cover_available})\n"

In [2]:
# 定义预测模型
from weather_data import WeatherData
from prediction import WeatherPrediction, YesterdaysWeather #, SimplePrediction, SophisticatedPrediction

class SimplePrediction(WeatherPrediction):
    """Simple prediction model, based on yesterday's weather."""

    def __init__(self, weather_data, days):
        """
        Parameters:
            weather_data (WeatherData): Collection of weather data.
            days (int): Days of data to use for making the prediction.

        Pre-condition:
            weather_data.size() > 0
        """
        super().__init__(weather_data)
        self.days = self._weather_data.size() if days > self._weather_data.size() else days
        self._weather_data = self._weather_data.get_data(self.days)

    def get_number_days(self):
        """(int) Number of days of data being used in prediction"""
        return self.days

    def chance_of_rain(self):
        """(int) Percentage indicating chance of rain occurring."""
        
        result = sum([i.get_rainfall() for i in self._weather_data])*9/self.days
        
        return 100 if result > 100 else result

    def high_temperature(self):
        """(float) Expected high temperature."""
        return max([i.get_high_temperature() for i in self._weather_data])

    def low_temperature(self):
        """(float) Expected low temperature."""
        return min([i.get_low_temperature() for i in self._weather_data])

    def humidity(self):
        """(int) Expected humidity."""
        return sum([i.get_humidity() for i in self._weather_data])/self.days

    def wind_speed(self):
        """(int) Expected average wind speed."""
        return sum([i.get_average_wind_speed() for i in self._weather_data])/self.days

    def cloud_cover(self):
        """(int) Expected amount of cloud cover."""
        return sum([i.get_cloud_cover() for i in self._weather_data])/self.days
    
class SophisticatedPrediction(WeatherPrediction):
    def __init__(self,weather_data, days):
        """
        Parameters:
            weather_data (WeatherData): Collection of weather data.
            days(int): Days of data to use for making the prediction.

        Pre-condition:
            weather_data.size() > 0
        """
        super().__init__(weather_data)
        self.days = self._weather_data.size() if days > self._weather_data.size() else days
        self._weather_data = self._weather_data.get_data(self.days)
        self.yesterday_air_pressure = self._weather_data[-1].get_air_pressure()
        self.average_air_pressure = sum([i.get_air_pressure() for i in self._weather_data])/self.days
        self.yesterday_wind_direction = self._weather_data[-1].get_wind_direction()

    def get_number_days(self):
        """ pass """
        return self.days

    def chance_of_rain(self):
        """ pass """
        average_rainfall = sum([i.get_chance_of_rain() for i in self._weather_data])/self.days
        
        # self._weather_data[-1] indicated yeasterday's air preasure
        if self.yesterday_air_pressure < self.average_air_pressure:
            average_rainfall = average_rainfall * 10
        elif self.yesterday_air_pressure > self.average_air_pressure:
            average_rainfall = average_rainfall * 7
        
        # self._weather_data[-4] indicated yesterday's wind direction
        if 'E' in self.yesterday_wind_direction:
            average_rainfall = average_rainfall * 1.2
        
        # >100
        if average_rainfall > 100:
            average_rainfall = 100
        
        return average_rainfall

    def high_temperature(self):
        average_high_temperature = sum([i.get_high_temperature() for i in self._weather_data])/self.days
        if self.yesterday_air_pressure > self.average_air_pressure:
            average_high_temperature += 2
        return average_high_temperature
    
    def low_temperature(self):
        average_low_temperature = sum([i.get_low_temperature() for i in self._weather_data])/self.days
        if self.yesterday_air_pressure < self.average_air_pressure:
            average_low_temperature -= 2
        return average_low_temperature
    
    def humidity(self):
        average_humidity = sum([i.get_humidity() for i in self._weather_data])/self.days
        if self.yesterday_air_pressure < self.average_air_pressure:
            average_humidity += 15
        if self.yesterday_air_pressure > self.average_air_pressure:
            average_humidity -= 15
        if average_humidity < 0:
            average_humidity = 0
        if average_humidity > 100:
            average_humidity = 100
        return average_humidity
    
    def cloud_cover(self):
        average_cloud_cover = sum([i.get_cloud_cover() for i in self._weather_data_])/self.days
        if self.yesterday_air_pressure < self.average_air_pressure:
            average_cloud_cover += 2
        if average_cloud_cover > 9:
            average_cloud_cover = 9
        return  average_cloud_cover
    
    def wind_speed(self):
        average_wind_speed = sum([i.get_wind_speed() for i in self._weather_data])/self.days
        # self._weather_data[-3] indacated yesterday wind speed
        if self.yesterday_wind_direction > 4 * average_wind_speed:
            average_wind_speed *= 1.2
        return  average_wind_speed

In [3]:
# 事件决策类
# 输入用户事件、预测模型，产出预测结果（-5~5）
class EventDecision(object):
    """Uses event details to decide if predicted weather suits an event."""

    def __init__(self, event, prediction_model):
        """
        Parameters:
            event (Event): The event to determine its suitability.
            prediction_model (WeatherPrediction): Specific prediction model.
                           An object of a subclass of WeatherPrediction used 
                           to predict the weather for the event.
        """
        self._event = event
        self._prediction_model = prediction_model

    def _temperature_factor(self):
        """
        Determines how advisable it is to continue with the event based on
        predicted temperature

        Return:
            (float) Temperature Factor
        """
        humidity = self._prediction_model.humidity()
        high_temperature = self._prediction_model.high_temperature()
        low_temperature = self._prediction_model.low_temperature()
        wind_speed = self._prediction_model.wind_speed()
        cloud_cover = self._prediction_model.cloud_cover()
        times = self._event.get_time()
        outdoors = self._event.get_outdoors()
        cover = self._event.get_cover_available()
        
        
        if humidity >70:
            humidity_factor = humidity/20
            if humidity_factor > 0:
                high_temperature += humidity_factor
                low_temperature += humidity_factor
            elif humidity_factor < 0:
                high_temperature -= humidity_factor
                low_temperature -= humidity_factor

        if (6 <= times <=19 and outdoors and high_temperature >= 30) or high_temperature >= 45:
            temp_factor = high_temperature/-5 + 6
            if temp_factor < 0:
                temp_factor += 1 if cover else 0
                temp_factor += 1 if 3 < wind_speed < 10 else 0 
                temp_factor += 1 if cloud_cover > 4 else 0
        elif (0 <= times <= 5 or 20 >= times>= 23) and low_temperature < 5 and high_temperature < 45:
            temp_factor = low_temperature/5 - 1.1
        elif low_temperature >15 and high_temperature < 30:
            temp_factor = (high_temperature-low_temperature)/5
        else:
            temp_factor = 0
        
        return temp_factor

    def _rain_factor(self):
        """
        Determines how advisable it is to continue with the event based on
        predicted rainfall

        Return:
            (float) Rain Factor
        """
        rain_pct = self._prediction_model.chance_of_rain()
        if rain_pct < 20:
            rain_factor = rain_pct/-5 + 4
        elif rain_pct > 50:
            rain_factor = rain_pct/-20 + 1
        else:
            rain_factor = 0
            
        wind_speed = self._prediction_model.wind_speed()
        if self._event.get_outdoors() and self._event.get_cover_available() and wind_speed < 5:
            rain_factor += 1
        if wind_speed > 15 and rain_factor < 2:
            rain_factor += (wind_speed/-15)
        if rain_factor < -9:
            rain_factor = - 9

        return rain_factor

    def advisability(self):
        """Determine how advisable it is to continue with the planned event.

        Return:
            (float) Value in range of -5 to +5,
                    -5 is very bad, 0 is neutral, 5 is very beneficial
        """
        score = self._temperature_factor() + self._rain_factor()
        if score > 5:
            score = 5
        elif score < -5:
            score = -5
        else:
            pass
        return score

In [4]:
# 定义用户交互界面
# 获取用户事件，获取用户选择模型，给客户返回预测模型，获取用户是否继续预测
class UserInteraction(object):
    """Simple textual interface to drive program."""

    def __init__(self):
        """
        """
        self._event = None
        self._prediction_model = None
        
    def _get_user_answer(self):
        """Parse the user's input.

        Return:
            (list): A list of user's input.
        """
        question_lists = [
            "What is the name of the event?",
            "Is the event outdoors?",
            "Is there covered shelter?",
            "What time is the event?"
        ]
        result = []
        for idx, i in enumerate(question_lists):
            value = input(f"{i}")
            if idx == 0:
                value = value.strip()
            elif idx in [1,2]:
                value = True if value.upper() in ['Y', 'YES'] else False
            else:
                value = int(value.strip())
            result.append(value)
            return result
        

    def get_event_details(self):
        """Prompt the user to enter details for an event.

        Return:
            (Event): An Event object containing the event details.
        """
        name, outdoors, cover_available, time = self._get_user_answer()
        self._event = Event(name, outdoors, cover_available, time)
        return self._event

    def get_prediction_model(self, weather_data):
        """Prompt the user to select the model for predicting the weather.

        Parameter:
            weather_data (WeatherData): Data used for predicting the weather.

        Return:
            (WeatherPrediction): Object of the selected prediction model.
        """
        print("Select the weather prediction model you wish to use:")
        print("  1) Yesterday's weather.")
        print("  2) Simple prediction.")
        print("  3) Sophisticated prediction.")
        model_choice = input("> ")
        # Error handling can be added to this method.
        if model_choice == '1' :
            self._prediction_model = YesterdaysWeather(weather_data)
        elif model_choice == '2':
            days = input('Enter how many days of data you wish to use for making the prediction:')
            self._prediction_model = SimplePrediction(weather_data, int(days.strip()))
        elif model_choice == '3':
            days = input('Enter how many days of data you wish to use for making the prediction:')
            self._prediction_model = SophisticatedPrediction(weather_data, int(days.strip()))
        else:
            print('error select, please input integer in 1~3')
            return self.get_prediction_model(weather_data)
        # Cater for other prediction models when they are implemented.
        return self._prediction_model

    def output_advisability(self, impact):
        """Output how advisable it is to go ahead with the event.

        Parameter:
            impact (float): Impact of the weather on the event.
                            -5 is very bad, 0 is neutral, 5 is very beneficial
        """
        # The following print statement is an example of printing out the
        # class name of an object, which you may use for making the
        # advisability output more meaningful.
        print(f"Based on the {type(self._prediction_model).__name__} model, the advisability of holding {self._event.get_name()} is {impact}.\n")

    def another_check(self):
        """Ask user if they want to check using another prediction model.

        Return:
            (bool): True if user wants to check using another prediction model.
        """
        user_choice = input("Would you like to check again? ").strip()
        return True if user_choice.upper() in ['Y', 'YES'] else False

In [5]:
# 处理主流程
def main():
    """Main application's starting point."""
    check_again = True  # 结束标志   
    weather_data = WeatherData()  # 实例化天气数据库容器
    weather_data.load("weather_data.csv")  # 装载天气数据
    user_interface = UserInteraction() # 实例化用户交互类，准备用户交互

    print("Let's determine how suitable your event is for the predicted weather.")
    event = user_interface.get_event_details() # 开始用户交互，获取四个关键参数，并使用参数实例化用户事件

    while check_again: # 进入循环，开启预测，每次检测是否停止预测，终止程序
        prediction_model = user_interface.get_prediction_model(weather_data) # 用户选择预测模型，并实例化
        decision = EventDecision(event, prediction_model)  # 装载用户事件及当前预测模型，实例化决策类
        impact = decision.advisability()  # 使用决策类进行决策，得到决策结果
        user_interface.output_advisability(impact)  # 将决策结果返回给用户
        check_again = user_interface.another_check() # 询问用户是否再次进行预测

In [6]:
main()

Let's determine how suitable your event is for the predicted weather.


What is the name of the event? xzw
Is the event outdoors? 1
Is there covered shelter? 1
What time is the event? 


ValueError: invalid literal for int() with base 10: ''

In [7]:
i=1
input(f"{i}")

1 1


'1'

In [10]:
question_list = [
    "What is the name of the event?",
    "Is the event outdoors?",
    "Is there covered shelter?",
    "What time is the event?"]
result = []
for index, i in enumerate(question_list):
    value = input(f"{i}")
    if index == 0:
        value = value.strip()
    elif index in [1,2]:
        value = True if value.upper in ("Y", "YES") else False
    else:
        value = int(value.strip())
    result.append(value)
print(result)

What is the name of the event? 1
Is the event outdoors? 2
Is there covered shelter? 3
What time is the event? 4


['1', False, False, 4]


In [4]:
a = 0 or 100
if 1:
    print('bingo')

bingo
